# 2D Puzzle Solver

This notebook solves a 2D grid puzzle where we need to:
1. Find pairs of number cells whose sum equals a target sum
2. Connect these pairs with non-overlapping paths

## Problem Constraints:
- Grid is n x n
- Some cells contain positive numbers (m cells, m is even)
- Paths can only move: left, right, up, down
- Paths cannot go through number cells (except start/end)
- Two paths cannot share the same empty cell

In [25]:
from typing import List, Tuple, Set, Optional, Dict

try:
    from ortools.sat.python import cp_model
    ORTOOLS_AVAILABLE = True
except ImportError:
    ORTOOLS_AVAILABLE = False
    print("Warning: OR-Tools not available. Install with: pip install ortools")

Coord = Tuple[int, int]

def solve_2d_puzzle(n: int, number_cells: List[Tuple[int, int, int]], target_sum: int) -> List[List[Tuple[int, int]]]:
    """
    Solve the 2D puzzle by finding pairs of number cells that sum to target_sum
    and connecting them with non-overlapping paths.
    
    Args:
        n: Grid size (n x n)
        number_cells: List of tuples (x, y, value) for cells with numbers
        target_sum: Target sum for pairs
        
    Returns:
        List of paths, where each path is a list of (x, y) coordinates.
        Returns empty list if no solution exists.
    """
    if not ORTOOLS_AVAILABLE:
        raise ImportError("OR-Tools is required. Install with: pip install ortools")
    
    # Create a grid to mark number cells (0 = empty, >0 = number cell)
    grid = [[0 for _ in range(n)] for _ in range(n)]
    
    for x, y, value in number_cells:
        grid[x][y] = value
    
    # Call the CP-SAT solver (adapted from solve_sum_pairs_disjoint_paths)
    result = find_valid_solution(grid, target_sum, time_limit_s=30.0)
    
    if result is None:
        return []
    
    # Convert from list of dicts with "path" key to list of paths
    return [item["path"] for item in result]

In [26]:
def find_valid_solution(
    grid: List[List[int]],
    target_sum: int,
    time_limit_s: float = 30.0,
    minimize_total_path_cells: bool = True,
) -> Optional[List[Dict]]:
    """
    Solve the puzzle using CP-SAT with flow-based path modeling.
    
    Args:
        grid: n x n grid where grid[r][c] == 0 means empty cell, >0 means numbered terminal
        target_sum: Target sum for pairs
        time_limit_s: Time limit for solver
        minimize_total_path_cells: Whether to minimize total path cells
        
    Returns:
        A list of solutions, one per selected pair, each dict has:
        { "pair": ((r1,c1,val1), (r2,c2,val2)), "path": [(r,c), ...] }
        or None if infeasible / not found within limit.
    """
    n = len(grid)
    assert all(len(row) == n for row in grid), "Grid must be n x n"
    
    # Use tolerance for floating point comparison
    tolerance = 1e-9
    
    # Collect terminals
    terminals: List[Tuple[Coord, float]] = []
    terminal_set = set()
    for r in range(n):
        for c in range(n):
            if grid[r][c] > 0 or abs(grid[r][c]) > tolerance:  # Handle float numbers
                terminals.append(((r, c), float(grid[r][c])))
                terminal_set.add((r, c))
    
    m = len(terminals)
    if m % 2 != 0:
        return None
    
    # Build candidate pairs whose values sum to target_sum
    candidates: List[Tuple[int, int]] = []
    for i in range(m):
        for j in range(i + 1, m):
            if abs(terminals[i][1] + terminals[j][1] - target_sum) < tolerance:
                candidates.append((i, j))
    
    if not candidates:
        return None
    
    # Quick feasibility: every terminal must have at least one candidate partner
    incident_count = [0] * m
    for i, j in candidates:
        incident_count[i] += 1
        incident_count[j] += 1
    if any(k == 0 for k in incident_count):
        return None
    
    # Precompute adjacency (4-neighbor moves) as directed arcs
    def neighbors(rc: Coord) -> List[Coord]:
        r, c = rc
        out = []
        if r > 0: out.append((r - 1, c))
        if r + 1 < n: out.append((r + 1, c))
        if c > 0: out.append((r, c - 1))
        if c + 1 < n: out.append((r, c + 1))
        return out
    
    all_cells: List[Coord] = [(r, c) for r in range(n) for c in range(n)]
    empty_cells: List[Coord] = [rc for rc in all_cells if rc not in terminal_set]
    
    model = cp_model.CpModel()
    
    # Decision: select which candidate pairs are used (perfect matching)
    use_pair = {}
    for e, (i, j) in enumerate(candidates):
        use_pair[e] = model.NewBoolVar(f"use_pair[{e}]")
    
    # Perfect matching: each terminal used exactly once
    for ti in range(m):
        involved = []
        for e, (i, j) in enumerate(candidates):
            if i == ti or j == ti:
                involved.append(use_pair[e])
        model.Add(sum(involved) == 1)
    
    # Path variables per candidate pair
    # f[e,u,v] for directed edge u->v (u and v adjacent)
    # x[e,c] for empty cell usage
    f = {}
    x = {}
    dist = {}
    
    # Upper bound for distance/order variables
    DMAX = n * n
    BIG_M = DMAX
    
    # Helper to create/get distance var
    def dist_var(e: int, cell: Coord):
        key = (e, cell)
        if key not in dist:
            dist[key] = model.NewIntVar(0, DMAX, f"dist[{e},{cell[0]},{cell[1]}]")
        return dist[key]
    
    for e, (i, j) in enumerate(candidates):
        s = terminals[i][0]
        t = terminals[j][0]
        
        # Create x on empty cells
        for cell in empty_cells:
            x[(e, cell)] = model.NewBoolVar(f"x[{e},{cell[0]},{cell[1]}]")
            # If pair not selected, cannot use cell
            model.Add(x[(e, cell)] <= use_pair[e])
        
        # Create directed edge vars on adjacency arcs
        for u in all_cells:
            for v in neighbors(u):
                f[(e, u, v)] = model.NewBoolVar(f"f[{e},{u[0]},{u[1]}->{v[0]},{v[1]}]")
                model.Add(f[(e, u, v)] <= use_pair[e])
        
        # No traversal through numbered cells other than s,t
        for term_rc, _val in terminals:
            if term_rc != s and term_rc != t:
                # force all incident edges to 0
                u = term_rc
                for v in neighbors(u):
                    model.Add(f[(e, u, v)] == 0)
                    model.Add(f[(e, v, u)] == 0)
        
        # Flow conservation constraints
        for u in all_cells:
            in_edges = [f[(e, v, u)] for v in neighbors(u)]
            out_edges = [f[(e, u, v)] for v in neighbors(u)]
            
            if u == s:
                # source: out - in == use_pair
                model.Add(sum(out_edges) - sum(in_edges) == use_pair[e])
            elif u == t:
                # sink: in - out == use_pair
                model.Add(sum(in_edges) - sum(out_edges) == use_pair[e])
            elif u in terminal_set:
                # other numbered cells already blocked; still ensure no flow
                model.Add(sum(in_edges) == 0)
                model.Add(sum(out_edges) == 0)
            else:
                # empty cell: either unused (0 in/out) or used (1 in and 1 out)
                # Enforce in == out
                model.Add(sum(in_edges) == sum(out_edges))
                # Degree at most 1 to prevent branching
                model.Add(sum(in_edges) <= 1)
                model.Add(sum(out_edges) <= 1)
                # Link to x: if used then exactly one in/out
                model.Add(sum(in_edges) == x[(e, u)])
                model.Add(sum(out_edges) == x[(e, u)])
        
        # Cycle elimination via distance/order constraints
        model.Add(dist_var(e, s) == 0)
        
        # For any used directed edge u->v, enforce dist[v] = dist[u] + 1
        for u in all_cells:
            for v in neighbors(u):
                edge = f[(e, u, v)]
                du = dist_var(e, u)
                dv = dist_var(e, v)
                model.Add(dv >= du + 1 - BIG_M * (1 - edge))
                model.Add(dv <= du + 1 + BIG_M * (1 - edge))
    
    # Vertex-disjointness on empty cells across all pairs
    for cell in empty_cells:
        model.Add(sum(x[(e, cell)] for e in range(len(candidates))) <= 1)
    
    # Objective (optional): minimize total number of used empty cells across all paths
    if minimize_total_path_cells:
        model.Minimize(sum(x.values()))
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = float(time_limit_s)
    
    status = solver.Solve(model)
    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return None
    
    # Extract selected pairs and reconstruct paths by following outgoing edges from source
    solutions = []
    
    for e, (i, j) in enumerate(candidates):
        if solver.Value(use_pair[e]) != 1:
            continue
        
        s = terminals[i][0]
        t = terminals[j][0]
        val_s = terminals[i][1]
        val_t = terminals[j][1]
        
        # Follow edges from s until t
        path = [s]
        cur = s
        visited = {s}
        while cur != t:
            next_nodes = []
            for v in neighbors(cur):
                if solver.Value(f[(e, cur, v)]) == 1:
                    next_nodes.append(v)
            if len(next_nodes) != 1:
                return None
            cur = next_nodes[0]
            if cur in visited:
                return None
            visited.add(cur)
            path.append(cur)
        
        solutions.append({
            "pair": ((s[0], s[1], val_s), (t[0], t[1], val_t)),
            "path": path
        })
    
    return solutions

## Test Cases

Let's test the solver with some examples:

In [27]:
# Visualization function
def visualize_solution(n: int, number_cells: List[Tuple[int, int, int]], 
                       paths: List[List[Tuple[int, int]]]):
    """Visualize the grid and paths"""
    grid = [['.' for _ in range(n)] for _ in range(n)]
    
    # Mark number cells
    for x, y, value in number_cells:
        grid[x][y] = str(value)
    
    # Mark paths
    for path_idx, path in enumerate(paths):
        for i, (x, y) in enumerate(path):
            if grid[x][y] == '.':
                # Use different symbols for different paths
                symbols = ['#', '*', '+', '=', '-', '~']
                grid[x][y] = symbols[path_idx % len(symbols)]
            elif i == 0 or i == len(path) - 1:
                # Start/end cells keep their numbers
                pass
    
    print("\nGrid visualization:")
    for row in grid:
        print(' '.join(f'{cell:>3}' for cell in row))


In [28]:
# Test Case 1: Simple 3x3 grid
n1 = 3
number_cells1 = [
    (0, 0, 5),  # Top-left: 5
    (2, 2, 3),  # Bottom-right: 3
]
target_sum1 = 8

result1 = solve_2d_puzzle(n1, number_cells1, target_sum1)
print("Test Case 1:")
print(f"Grid size: {n1}x{n1}")
print(f"Number cells: {number_cells1}")
print(f"Target sum: {target_sum1}")
print(f"Result: {result1}")
if result1:
    for i, path in enumerate(result1):
        print(f"  Path {i+1}: {path}")
    # Visualize the solution
    visualize_solution(n1, number_cells1, result1)
else:
    print("  No solution found")

Test Case 1:
Grid size: 3x3
Number cells: [(0, 0, 5), (2, 2, 3)]
Target sum: 8
Result: [[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)]]
  Path 1: [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2)]

Grid visualization:
  5   .   .
  #   #   .
  .   #   3


In [30]:
# Test Case 2: 4x4 grid with multiple pairs (designed to have non-overlapping paths)
n2 = 4
number_cells2 = [
    (0, 0, 2),  # Top-left
    (3, 0, 5),  # Bottom-left (2+5=7)
    (0, 3, 4),  # Top-right
    (3, 3, 3),  # Bottom-right (4+3=7)
]
target_sum2 = 7  # 2+5=7, 4+3=7

result2 = solve_2d_puzzle(n2, number_cells2, target_sum2)
print("\nTest Case 2:")
print(f"Grid size: {n2}x{n2}")
print(f"Number cells: {number_cells2}")
print(f"Target sum: {target_sum2}")
print(f"Result: {result2}")
if result2:
    for i, path in enumerate(result2):
        print(f"  Path {i+1}: {path}")
    # Visualize the solution
    visualize_solution(n2, number_cells2, result2)
else:
    print("  No solution found")


Test Case 2:
Grid size: 4x4
Number cells: [(0, 0, 2), (3, 0, 5), (0, 3, 4), (3, 3, 3)]
Target sum: 7
Result: [[(0, 0), (1, 0), (2, 0), (3, 0)], [(0, 3), (1, 3), (2, 3), (3, 3)]]
  Path 1: [(0, 0), (1, 0), (2, 0), (3, 0)]
  Path 2: [(0, 3), (1, 3), (2, 3), (3, 3)]

Grid visualization:
  2   .   .   4
  #   .   .   *
  #   .   .   *
  5   .   .   3


In [31]:
# Test Case 3: More complex example
n3 = 5
number_cells3 = [
    (0, 0, 1),
    (0, 4, 2),
    (4, 0, 3),
    (4, 4, 4),
]
target_sum3 = 5  # 1+4=5, 2+3=5

result3 = solve_2d_puzzle(n3, number_cells3, target_sum3)
print("\nTest Case 3:")
print(f"Grid size: {n3}x{n3}")
print(f"Number cells: {number_cells3}")
print(f"Target sum: {target_sum3}")
print(f"Result: {result3}")
if result3:
    for i, path in enumerate(result3):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n3, number_cells3, result3)
else:
    print("  No solution found")


Test Case 3:
Grid size: 5x5
Number cells: [(0, 0, 1), (0, 4, 2), (4, 0, 3), (4, 4, 4)]
Target sum: 5
Result: []
  No solution found


In [32]:
# Test Case 4: No solution case (pairs exist but paths overlap)
n4 = 3
number_cells4 = [
    (0, 0, 1),
    (0, 2, 2),
    (2, 0, 3),
    (2, 2, 4),
]
target_sum4 = 5  # 1+4=5, 2+3=5

result4 = solve_2d_puzzle(n4, number_cells4, target_sum4)
print("\nTest Case 4:")
print(f"Grid size: {n4}x{n4}")
print(f"Number cells: {number_cells4}")
print(f"Target sum: {target_sum4}")
print(f"Result: {result4}")
if result4:
    for i, path in enumerate(result4):
        print(f"  Path {i+1}: {path}")
    visualize_solution(n4, number_cells4, result4)
else:
    print("  No solution found (as expected if paths would overlap)")


Test Case 4:
Grid size: 3x3
Number cells: [(0, 0, 1), (0, 2, 2), (2, 0, 3), (2, 2, 4)]
Target sum: 5
Result: []
  No solution found (as expected if paths would overlap)


In [33]:
# Test Case 5: More complex example
n5 = 9
number_cells5 = [
    (0, 0, 2.3),
    (1, 7, 2.8),
    (3, 4, 1.3),
    (3, 6, 1.2),
    (4, 0, 3.8),
    (7, 1, 2.2),
    (8, 5, 3.7),
    (8, 8, 2.7),
]
target_sum5 = 5  # 1+4=5, 2+3=5

result5 = solve_2d_puzzle(n5, number_cells5, target_sum3)
print("\nTest Case 5:")
print(f"Grid size: {n5}x{n5}")
print(f"Number cells: {number_cells5}")
print(f"Target sum: {target_sum5}")
print(f"Result: {result5}")
if result5:
    for i, path in enumerate(result5):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n5, number_cells5, result5)
else:
    print("  No solution found")


Test Case 5:
Grid size: 9x9
Number cells: [(0, 0, 2.3), (1, 7, 2.8), (3, 4, 1.3), (3, 6, 1.2), (4, 0, 3.8), (7, 1, 2.2), (8, 5, 3.7), (8, 8, 2.7)]
Target sum: 5
Result: [[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)], [(1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (4, 2), (5, 2), (6, 2), (7, 2), (7, 1)], [(3, 4), (4, 4), (5, 4), (6, 4), (6, 5), (7, 5), (8, 5)], [(3, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (2, 2), (2, 1), (3, 1), (4, 1), (4, 0)]]
  Path 1 (length 17): [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)]
  Path 2 (length 17): [(1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (4, 2), (5, 2), (6, 2), (7, 2), (7, 1)]
  Path 3 (length 7): [(3, 4), (4, 4), (5, 4), (6, 4), (6, 5), (7, 5), (8, 5)]
 

In [34]:
# Test Case 6: More complex example
n6 = 9
number_cells6 = [
    (0, 0, 2.3),
    (1, 7, 2.8),
    (2, 1, 4.8),
    (3, 4, 1.3),
    (3, 6, 1.2),
    (4, 0, 3.8),
    (7, 1, 2.2),
    (7, 6, 0.2),
    (8, 5, 3.7),
    (8, 8, 2.7),
]
target_sum6 = 5  # 1+4=5, 2+3=5

result6 = solve_2d_puzzle(n6, number_cells6, target_sum6)
print("\nTest Case 6:")
print(f"Grid size: {n6}x{n6}")
print(f"Number cells: {number_cells6}")
print(f"Target sum: {target_sum6}")
print(f"Result: {result6}")
if result6:
    for i, path in enumerate(result6):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n6, number_cells6, result6)
else:
    print("  No solution found")


Test Case 6:
Grid size: 9x9
Number cells: [(0, 0, 2.3), (1, 7, 2.8), (2, 1, 4.8), (3, 4, 1.3), (3, 6, 1.2), (4, 0, 3.8), (7, 1, 2.2), (7, 6, 0.2), (8, 5, 3.7), (8, 8, 2.7)]
Target sum: 5
Result: [[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)], [(1, 7), (2, 7), (3, 7), (4, 7), (4, 6), (4, 5), (3, 5), (2, 5), (2, 4), (2, 3), (3, 3), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (7, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (5, 0), (6, 0), (7, 0), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (7, 4), (7, 5), (7, 6)], [(3, 4), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (8, 6), (8, 5)], [(3, 6), (2, 6), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0)]]
  Path 1 (length 17): [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 8), (2, 8), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8)]
  Path 2 (length 17): [(1, 7), (2, 7), (3, 7), (4, 7), (4,

In [35]:
# Test Case 7: More complex example
n7 = 8
number_cells7 = [
  (0, 0, 2.3),
  (1, 6, 2.5),
  (2, 1, 4.5),
  (3, 3, 1.3),
  (3, 5, 1.2),
  (4, 0, 3.8),
  (6, 1, 2.5),
  (6, 5, 0.5),
  (7, 4, 3.7),
  (7, 7, 2.7),
]
target_sum7 = 5

result7 = solve_2d_puzzle(n7, number_cells7, target_sum7)
print("\nTest Case 7:")
print(f"Grid size: {n7}x{n7}")
print(f"Number cells: {number_cells7}")
print(f"Target sum: {target_sum7}")
print(f"Result: {result7}")
if result7:
    for i, path in enumerate(result7):
        print(f"  Path {i+1} (length {len(path)}): {path}")
    visualize_solution(n7, number_cells7, result7)
else:
    print("  No solution found")




Test Case 7:
Grid size: 8x8
Number cells: [(0, 0, 2.3), (1, 6, 2.5), (2, 1, 4.5), (3, 3, 1.3), (3, 5, 1.2), (4, 0, 3.8), (6, 1, 2.5), (6, 5, 0.5), (7, 4, 3.7), (7, 7, 2.7)]
Target sum: 5
Result: [[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7)], [(1, 6), (2, 6), (3, 6), (4, 6), (4, 5), (4, 4), (3, 4), (2, 4), (2, 3), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (6, 1)], [(2, 1), (3, 1), (4, 1), (5, 1), (5, 0), (6, 0), (7, 0), (7, 1), (7, 2), (7, 3), (6, 3), (6, 4), (6, 5)], [(3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (6, 6), (7, 6), (7, 5), (7, 4)], [(3, 5), (2, 5), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0)]]
  Path 1 (length 15): [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7)]
  Path 2 (length 15): [(1, 6), (2, 6), (3, 6), (4, 6), (4, 5), (4, 4), (3, 4), (2, 4), (2, 3), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (6,

## Algorithm Explanation

The solution uses a backtracking approach:

1. **Find Valid Pairs**: Identify all pairs of number cells whose sum equals the target sum
2. **Generate Matchings**: Find all possible ways to partition number cells into pairs (perfect matching)
3. **Path Finding**: For each matching, use BFS to find paths between pairs
4. **Conflict Resolution**: Ensure paths don't share empty cells by tracking used cells
5. **Backtracking**: If a path combination fails, backtrack and try another matching

The algorithm ensures:
- All number cells are used exactly once
- Paths only use empty cells (except start/end)
- No two paths share the same empty cell
- Paths only move in 4 directions (up, down, left, right)

In [23]:
from typing import List, Tuple, Dict, Optional
from ortools.sat.python import cp_model


Coord = Tuple[int, int]


def solve_sum_pairs_disjoint_paths(
    grid: List[List[int]],
    target_sum: int,
    time_limit_s: float = 10.0,
    minimize_total_path_cells: bool = True,
) -> Optional[List[Dict]]:
    """
    Solve the puzzle:
      - grid is n x n
      - grid[r][c] == 0 means empty cell
      - grid[r][c] > 0 means a numbered terminal cell
      - pair terminals whose values sum to target_sum
      - connect each pair with a vertex-disjoint 4-neighbor path
      - paths cannot pass through any numbered cell except their own endpoints

    Returns:
      A list of solutions, one per selected pair, each dict has:
        { "pair": ((r1,c1,val1), (r2,c2,val2)), "path": [(r,c), ...] }
      or None if infeasible / not found within limit.
    """
    n = len(grid)
    assert all(len(row) == n for row in grid), "Grid must be n x n"

    # Collect terminals
    terminals: List[Tuple[Coord, int]] = []
    terminal_set = set()
    for r in range(n):
        for c in range(n):
            if grid[r][c] > 0:
                terminals.append(((r, c), grid[r][c]))
                terminal_set.add((r, c))

    m = len(terminals)
    if m % 2 != 0:
        raise ValueError(f"Number of numbered cells must be even, got {m}")

    # Build candidate pairs whose values sum to target_sum
    # We'll index terminals by i = 0..m-1
    candidates: List[Tuple[int, int]] = []
    for i in range(m):
        for j in range(i + 1, m):
            if terminals[i][1] + terminals[j][1] == target_sum:
                candidates.append((i, j))

    if not candidates:
        return None

    # Quick feasibility: every terminal must have at least one candidate partner
    incident_count = [0] * m
    for i, j in candidates:
        incident_count[i] += 1
        incident_count[j] += 1
    if any(k == 0 for k in incident_count):
        return None

    # Precompute adjacency (4-neighbor moves) as directed arcs
    def neighbors(rc: Coord) -> List[Coord]:
        r, c = rc
        out = []
        if r > 0: out.append((r - 1, c))
        if r + 1 < n: out.append((r + 1, c))
        if c > 0: out.append((r, c - 1))
        if c + 1 < n: out.append((r, c + 1))
        return out

    all_cells: List[Coord] = [(r, c) for r in range(n) for c in range(n)]
    empty_cells: List[Coord] = [rc for rc in all_cells if rc not in terminal_set]

    model = cp_model.CpModel()

    # Decision: select which candidate pairs are used (perfect matching)
    use_pair = {}
    for e, (i, j) in enumerate(candidates):
        use_pair[e] = model.NewBoolVar(f"use_pair[{e}]")

    # Perfect matching: each terminal used exactly once
    for ti in range(m):
        involved = []
        for e, (i, j) in enumerate(candidates):
            if i == ti or j == ti:
                involved.append(use_pair[e])
        model.Add(sum(involved) == 1)

    # Path variables per candidate pair
    # f[e,u,v] for directed edge u->v (u and v adjacent)
    # x[e,c] for empty cell usage
    f = {}
    x = {}
    dist = {}

    # Upper bound for distance/order variables
    # A simple path can't exceed n*n steps.
    DMAX = n * n
    BIG_M = DMAX

    # Helper to create/get distance var
    def dist_var(e: int, cell: Coord):
        key = (e, cell)
        if key not in dist:
            dist[key] = model.NewIntVar(0, DMAX, f"dist[{e},{cell[0]},{cell[1]}]")
        return dist[key]

    for e, (i, j) in enumerate(candidates):
        s = terminals[i][0]
        t = terminals[j][0]

        # Create x on empty cells
        for cell in empty_cells:
            x[(e, cell)] = model.NewBoolVar(f"x[{e},{cell[0]},{cell[1]}]")
            # If pair not selected, cannot use cell
            model.Add(x[(e, cell)] <= use_pair[e])

        # Create directed edge vars on adjacency arcs
        for u in all_cells:
            for v in neighbors(u):
                f[(e, u, v)] = model.NewBoolVar(f"f[{e},{u[0]},{u[1]}->{v[0]},{v[1]}]")
                model.Add(f[(e, u, v)] <= use_pair[e])

        # No traversal through numbered cells other than s,t
        for term_rc, _val in terminals:
            if term_rc != s and term_rc != t:
                # force all incident edges to 0
                u = term_rc
                for v in neighbors(u):
                    model.Add(f[(e, u, v)] == 0)
                    model.Add(f[(e, v, u)] == 0)

        # Flow conservation constraints
        for u in all_cells:
            in_edges = [f[(e, v, u)] for v in neighbors(u)]
            out_edges = [f[(e, u, v)] for v in neighbors(u)]

            if u == s:
                # source: out - in == use_pair
                model.Add(sum(out_edges) - sum(in_edges) == use_pair[e])
            elif u == t:
                # sink: in - out == use_pair
                model.Add(sum(in_edges) - sum(out_edges) == use_pair[e])
            elif u in terminal_set:
                # other numbered cells already blocked; still ensure no flow
                model.Add(sum(in_edges) == 0)
                model.Add(sum(out_edges) == 0)
            else:
                # empty cell: either unused (0 in/out) or used (1 in and 1 out)
                # Enforce in == out
                model.Add(sum(in_edges) == sum(out_edges))
                # Degree at most 1 to prevent branching
                model.Add(sum(in_edges) <= 1)
                model.Add(sum(out_edges) <= 1)
                # Link to x: if used then exactly one in/out
                # (If x=0, in/out can still be 0 due to <=1 and equality)
                model.Add(sum(in_edges) == x[(e, u)])
                model.Add(sum(out_edges) == x[(e, u)])

        # Cycle elimination via distance/order constraints
        # Fix dist[s] = 0 when selected; easiest: dist[s] == 0 always.
        model.Add(dist_var(e, s) == 0)

        # For any used directed edge u->v, enforce dist[v] = dist[u] + 1
        # Big-M relaxation when edge is 0.
        for u in all_cells:
            for v in neighbors(u):
                edge = f[(e, u, v)]
                du = dist_var(e, u)
                dv = dist_var(e, v)
                model.Add(dv >= du + 1 - BIG_M * (1 - edge))
                model.Add(dv <= du + 1 + BIG_M * (1 - edge))

        # Optional: if pair not selected, avoid forcing a dummy distance structure.
        # dist variables are already relaxable because edges are 0 when not selected.

    # Vertex-disjointness on empty cells across all pairs
    for cell in empty_cells:
        model.Add(sum(x[(e, cell)] for e in range(len(candidates))) <= 1)

    # Objective (optional): minimize total number of used empty cells across all paths
    if minimize_total_path_cells:
        model.Minimize(sum(x.values()))

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = float(time_limit_s)
    # Good defaults; you can uncomment for more parallelism if available:
    # solver.parameters.num_search_workers = 8

    status = solver.Solve(model)
    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return None

    # Extract selected pairs and reconstruct paths by following outgoing edges from source
    solutions = []

    # Build quick edge lookup for extraction
    for e, (i, j) in enumerate(candidates):
        if solver.Value(use_pair[e]) != 1:
            continue

        s = terminals[i][0]
        t = terminals[j][0]
        val_s = terminals[i][1]
        val_t = terminals[j][1]

        # Follow edges from s until t
        path = [s]
        cur = s
        visited = {s}
        while cur != t:
            next_nodes = []
            for v in neighbors(cur):
                if solver.Value(f[(e, cur, v)]) == 1:
                    next_nodes.append(v)
            if len(next_nodes) != 1:
                # Should not happen if constraints are correct, but guard anyway
                return None
            cur = next_nodes[0]
            if cur in visited:
                # cycle guard
                return None
            visited.add(cur)
            path.append(cur)

        solutions.append({
            "pair": ((s[0], s[1], val_s), (t[0], t[1], val_t)),
            "path": path
        })

    return solutions


In [24]:
# Example usage:
# 0 = empty, >0 = terminal
grid = [ 
    [2.3, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 2.5, 0], 
    [0, 4.5, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 1.3, 0, 1.2, 0, 0], 
    [3.8, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 2.5, 0, 0, 0, 0.5, 0, 0], 
    [0, 0, 0, 0, 3.7, 0, 0, 2.7], 
]
target = 5
sol = solve_sum_pairs_disjoint_paths(grid, target, time_limit_s=5.0)
if sol is None:
    print("No solution found.")
else:
    for item in sol:
        print(item["pair"], "->", item["path"])

((0, 0, 2.3), (7, 7, 2.7)) -> [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7), (7, 7)]
((1, 6, 2.5), (6, 1, 2.5)) -> [(1, 6), (2, 6), (3, 6), (4, 6), (4, 5), (4, 4), (3, 4), (2, 4), (2, 3), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (6, 1)]
((2, 1, 4.5), (6, 5, 0.5)) -> [(2, 1), (3, 1), (4, 1), (5, 1), (5, 0), (6, 0), (7, 0), (7, 1), (7, 2), (7, 3), (6, 3), (6, 4), (6, 5)]
((3, 3, 1.3), (7, 4, 3.7)) -> [(3, 3), (4, 3), (5, 3), (5, 4), (5, 5), (5, 6), (6, 6), (7, 6), (7, 5), (7, 4)]
((3, 5, 1.2), (4, 0, 3.8)) -> [(3, 5), (2, 5), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (3, 0), (4, 0)]
